autor: @LuisFalva

#### *SMOTE* es una técnica para balancear datos. Normalmente, a la hora de entrenar un modelo tenemos que generar nuestra variable *target* [0,1] con la cual podremos calcular una predicción a partir de los registros observados, ¿pero que pasa cuando el 'target' que nos interesa es la clase minoritaria? Esto es un problema típico que muchos modelos sufren, dado que nuestra clase de interés será, en la mayoría de los casos, la clase minoritaria, tenemos que buscar una técnica para implementar un sobremuestreo sin perder información.

<img src="src/smote.gif" width="750" align="center">

**img link: [The main issue with identifying Financial Fraud using Machine Learning (and how to address it)](https://towardsdatascience.com/the-main-issue-with-identifying-financial-fraud-using-machine-learning-and-how-to-address-it-3b1bf8fa1e0c)**

#### Dentro de este notebook, están las notas de estudio respecto a la técnica Synthetic Minority Oversampling Technique [SMOTE] la cual hace uso del algoritmo de k-NN para encontrar los vecinos más cercanos a la clase minoritaria, i.e. la clase de los positivos '1'. Se involucra el uso mixto de sklearn y pyspark, la idea de esta solución es publicar una forma de tantas posibles para implementar un método de muestreo de forma distribuida, por tal razón tenemos que definir a continuación la sesión de Spark.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

In [2]:
spark = SparkSession.builder.appName("SMOTE").getOrCreate()

**NOTA IMPORTANTE: Para hacer uso de la función *smote_samplig()* solo tenemos que importar la clase SparkSmote y crear la instancia de la clase:**

In [3]:
from smote.spark_smote import SparkSmote

spark_smote = SparkSmote()

#### Para la construcción de la función que nos ayudará a generar nuestras muestras sintéticas, vamos a cargar la tabla **"src/data/"**, la cual contiene una cantidad de variables que describen las caracteristicas principales de un cliente por cada renglón. El dataframe que usaremos mantendrá de origen las siguientes variables numéricas:
- **[age, child, saving, insight, backup, marital]**

In [4]:
arr_col = ["age", "child", "saving", "insight", "backup", "marital"]
smote_test = spark.read.parquet("src/data/").select(*arr_col)
smote_test.show(5, False)

+---+-----+------+-------+------+-------+
|age|child|saving|insight|backup|marital|
+---+-----+------+-------+------+-------+
|59 |1    |0     |1      |1     |married|
|56 |0    |1     |0      |1     |married|
|41 |1    |1     |0      |0     |married|
|55 |1    |0     |0      |1     |married|
|54 |1    |0     |0      |1     |married|
+---+-----+------+-------+------+-------+
only showing top 5 rows



**El set de datos contiene la variable 'marital', en dicha variable existen 3 clases: [married, single, divorced], por conveniencia al ejemplo que se contruye a lo largo de este notebook, se ha elegido la clase 'divorced' como nuestro target, podemos notar claramente que existe un desbalance considerable a la hora de contar la cantidad de registros que satisfacen nuestro público target, i.e. la clase 1.**

In [5]:
test = smote_test.select("*", (when(col("marital") == "divorced", 1).otherwise(0)).alias("target")).drop("marital")
test.groupBy("target").count().show()
test.where(col("target") == 1).show(5)

+------+-----+
|target|count|
+------+-----+
|     1| 1293|
|     0| 9869|
+------+-----+

+---+-----+------+-------+------+------+
|age|child|saving|insight|backup|target|
+---+-----+------+-------+------+------+
| 60|    0|     1|      0|     0|     1|
| 35|    0|     1|      1|     1|     1|
| 49|    1|     1|      1|     0|     1|
| 28|    0|     0|      0|     0|     1|
| 43|    1|     1|      0|     1|     1|
+---+-----+------+-------+------+------+
only showing top 5 rows



**Ahora bien, lo que se busca para entrenar un modelo de k vecinos cercanos [i.e. k-NN], por ejemplo, es un objeto de tipo numpy.array con los valores de cada registro, algo similar a esto:**

In [6]:
import numpy as np

np.array(test.where(col("target") == 1).drop("target").collect())

array([[60,  0,  1,  0,  0],
       [35,  0,  1,  1,  1],
       [49,  1,  1,  1,  0],
       ...,
       [52,  0,  0,  0,  0],
       [38,  0,  1,  0,  1],
       [60,  1,  1,  0,  1]])

**NOTA: Sin embargo, para convertir de un Spark Dataframe a un objeto de tipo numpy.array es conveniente antes transformarlo a RDD, por lo que los métodos de la clase SparkSMOTE se encargarán de realizar internamente esos parseos.**

#### El algoritmo de k-NN necesita recibir como entrada un objeto de tipo np.array por lo que se debe convertir nuestro spark Dataframe a un objeto de tipo numpy array, y para ello tenemos que convertir nuestro spark dataframe a rdd's, para que la estructura de datos al ser transformada ésta sea de manera distribuida.

#### Para generar nuestras muestras sintéticas debemos antes vectorizar los atributos que tengamos en nuestra tabla de datos, esto significa que debemos tomar los valores de cada columna y crear vectores de longitud **$p$**. El método **smote_sampling** asume tres principales puntos:

- Normalización y estandarización de variables
- Mapeo de cada valor por columna a codificaciones binarias (StringIndexer, OneHotEncoder)
- Spark Dataframe vectorizado, i.e. con columna de vectores densos y escasos (features), y columna dicotómica (label)

In [7]:
vector_assemble = spark_smote.vector_assembling(test, "target")
vector_assemble.show(5, False)

+----------------------+-----+
|features              |label|
+----------------------+-----+
|[59.0,1.0,0.0,1.0,1.0]|0    |
|[56.0,0.0,1.0,0.0,1.0]|0    |
|[41.0,1.0,1.0,0.0,0.0]|0    |
|[55.0,1.0,0.0,0.0,1.0]|0    |
|[54.0,1.0,0.0,0.0,1.0]|0    |
+----------------------+-----+
only showing top 5 rows



#### Como muestra de su funcionamiento, para aplicar el método *smote_sampling* requerimos de la tabla anterior con variables previamente standarizados, codificados y vectorizados. Como se puede ver, el método recibe los argumentos 'pct_over_min' y 'pct_under_max' configurados por default en [100, 100] respectivamente, cada uno de esos argumentos ayudarán a manipular el submuestreo o sobremuestreo de ambas clases que se ven en la siguiente tabla.

- **pct_over_min; modificará la cantidad de registros que existe para la clase minoritaria sobremuestreando los registros con valores sintéticos, en este caso, la clase '1'**

- **pct_under_max; modificará la cantidad de registros que existe para la clase mayoritaria submuestreando los registros, en este caso, la clase '0'**

In [8]:
smote_sample = spark_smote.smote_sampling(spark, vector_assemble, pct_over_min=600, pct_under_max=100)
smote_sample.groupBy("label").count().show()
smote_sample.where(col("label") == 1).orderBy(col("features").desc()).limit(5).toPandas()

+-----+-----+
|label|count|
+-----+-----+
|    0| 9869|
|    1| 9051|
+-----+-----+



,features,label
0,"[1236.778682061702, 1235.37473148036, 1235.414...",1
1,"[1233.0239286735316, 1232.6035057438876, 1232....",1
2,"[1231.2575290198367, 1231.1173686630991, 1231....",1
3,"[1229.467505661976, 1229.0195860538968, 1229.0...",1
4,"[1223.9368066679888, 1222.459148073263, 1222.4...",1


#### Referencias:
- https://rikunert.com/SMOTE_explained
- https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-14-106
- https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/